# FSRS4Anki Scheduler Implementation

In [184]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler


In [185]:
# Necessary Input:
# z_score - only for questions that have been asked
# asked questions to put into scheduler

# parameters for FSRS
w = [1.1008, 1.2746, 5.7619, 10.5114, 5.3148, 1.5796, 1.244, 0.003, 1.5741, 0.1741, 1.0137, 2.7279, 0.0114, 0.3071, 0.3981, 0.0, 1.9569]
requestRetention = 0.82  # recommended setting: 0.8 ~ 0.9

# parameters for Anki
graduatingInterval = 1
easyInterval = 4
easyBonus = 1.3
hardInterval = 1.2
intervalModifier = 1
newInterval = 0
minimumInterval = 1
leechThreshold = 8
leechSuspend = False

# common parameters
maximumInterval = 36500
new_cards_limits = 20
review_limits = 400
max_time_limts = 10000
learn_days = 50

# How many questions have we asked?
deck_size = 1000

# get the true time from review logs
filename = "collection-2022-09-18@13-21-58.colpkg"

# smooth curves
moving_average_period = 14

# Set it to True if you don't want the optimizer to use the review logs from suspended cards.
filter_out_suspended_cards = False

# Red: 1, Orange: 2, Green: 3, Blue: 4, Pink: 5, Turquoise: 6, Purple: 7
# Set it to [1, 2] if you don't want the optimizer to use the review logs from cards with red or orange flag.
filter_out_flags = []

In [186]:
#df = pd.read_pickle("sample_data.pkl")


In [187]:
#df = pd.read_csv("data.csv", columns = ["id, review_time, review_state, z_scores"])
# data = {
#     "id": [1, 2, 3, 4, 5],
#     "review_time": [[1, 5, 10, 15], [20, 25, 30], [5], [3, 7], [2, 6, 10, 14]],
#     "review_state": [[0, 1, 1, 1], [3, 2, 1], [2], [0, 1], [1, 2, 3, 2]],
#     "z_scores": [0.001, 0.3, 0.5, 0.7, 0.9]
# }
data = {'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 'review_time': [[1, 5, 10, 15],
  [20, 25, 30],
  [5],
  [3, 7],
  [2, 6, 10, 14],
  [22, 17, 5],
  [5, 18, 6],
  [28, 24, 30, 17],
  [12],
  [18],
  [20, 13, 17, 13],
  [27, 25, 16],
  [1, 11, 28, 2],
  [16, 16],
  [8]],
 'review_state': [[0, 1, 1, 1],
  [3, 2, 1],
  [2],
  [0, 1],
  [1, 2, 3, 2],
  [0, 1, 2],
  [0, 1, 2],
  [0, 1, 2, 3],
  [0],
  [0],
  [0, 1, 2, 3],
  [0, 1, 2],
  [0, 1, 2, 3],
  [0, 1],
  [0]],
 'z_scores': [1,
  0.3,
  0.5,
  0.7,
  0.9,
  0.089,
  0.088,
  0.945,
  0.414,
  0.469,
  0.888,
  0.986,
  0.159,
  0.944,
  0.886]}
df = pd.DataFrame(data)
deck_size = len(df)

def calculate_review_duration(states, times):
    if states[-1] != 2:
        return 5
    else:
        # Find the most recent transition to state 2 from either 1 or 3
        for i in range(len(states) - 1, 0, -1):
            if states[i] == 2 and (states[i - 1] == 1 or states[i - 1] == 3):
                return times[i] - times[i - 1]
        return 5  # Default value if no valid transition is found

# Apply the function to each row
df['review_duration'] = [calculate_review_duration(s, t) for s, t in zip(df['review_state'], df['review_time'])]

# Define the bins for the intervals
bins = [0.25, 0.5, 0.75]

# Use numpy's digitize method to convert z_scores to review_rating
df['review_rating'] = np.digitize(df['z_scores'], bins) + 1
df['review_rating'] = 5 - df['review_rating']
df['review_time_curr'] = df['review_time'].apply(lambda x: x[-1])
df['review_state_curr'] = df['review_state'].apply(lambda x: x[-1])
df.head()

,id,review_time,review_state,z_scores,review_duration,review_rating,review_time_curr,review_state_curr
0,1,"[1, 5, 10, 15]","[0, 1, 1, 1]",1.0,5,1,15,1
1,2,"[20, 25, 30]","[3, 2, 1]",0.3,5,3,30,1
2,3,[5],[2],0.5,5,2,5,2
3,4,"[3, 7]","[0, 1]",0.7,5,2,7,1
4,5,"[2, 6, 10, 14]","[1, 2, 3, 2]",0.9,4,1,14,2


In [188]:
New = 0
Learning = 1
Review = 2
Relearning = 3

df.sort_values(by=["id", "review_time_curr"], inplace=True, ignore_index=True)

# new_card_revlog = df[
#     (df["review_state_curr"] == New) & (df["review_rating"].isin([1, 2, 3, 4]))
# ]
# first_rating_prob = np.zeros(4)
# first_rating_prob[new_card_revlog["review_rating"].value_counts().index - 1] = (
#     new_card_revlog["review_rating"].value_counts()
#     / new_card_revlog["review_rating"].count()
# )
# print(first_rating_prob)
recall_card_revlog = df[
    (df["review_state_curr"] == Review) & (df["review_rating"].isin([2, 3, 4]))
]
review_rating_prob = np.zeros(3)
review_rating_prob[recall_card_revlog["review_rating"].value_counts().index - 2] = (
    recall_card_revlog["review_rating"].value_counts()
    / recall_card_revlog["review_rating"].count()
)
random_array = np.random.rand(4)
random_array /= random_array.sum()
first_rating_prob = random_array




df["review_state_curr"] = df["review_state_curr"].map(
    lambda x: x if x != New else Learning)

recall_costs = np.zeros(3)
recall_costs_df = recall_card_revlog.groupby(by="review_rating")[
    "review_duration"
].mean()
recall_costs[recall_costs_df.index - 2] = recall_costs_df / 1000

state_sequence = np.array(df["review_state_curr"])
duration_sequence = np.array(df["review_duration"])
learn_cost = round(
    df[df["review_state_curr"] == Learning]["review_duration"].sum()
    / len(df["id"].unique())
    / 1000,
    1,
)

state_block = dict()
state_count = dict()
state_duration = dict()
last_state = state_sequence[0]
state_block[last_state] = 1
state_count[last_state] = 1
state_duration[last_state] = duration_sequence[0]
for i, state in enumerate(state_sequence[1:]):
    state_count[state] = state_count.setdefault(state, 0) + 1
    state_duration[state] = state_duration.setdefault(
        state, 0) + duration_sequence[i]
    if state != last_state:
        state_block[state] = state_block.setdefault(state, 0) + 1
    last_state = state

recall_cost = round(state_duration[Review] / state_count[Review] / 1000, 1)

if Relearning in state_count and Relearning in state_block:
    forget_cost = round(
        state_duration[Relearning] /
        state_block[Relearning] / 1000 + recall_cost,
        1,
    )

print(f"average time for failed reviews: {forget_cost}s")
print(f"average time for recalled reviews: {recall_cost}s")
print(
    "average time for `hard`, `good` and `easy` reviews: %.1fs, %.1fs, %.1fs"
    % tuple(recall_costs)
)
print(f"average time for learning a new card: {learn_cost}s")
print(
    "Ratio of `hard`, `good` and `easy` ratings for recalled reviews: %.2f, %.2f, %.2f"
    % tuple(review_rating_prob)
)
print(
    "Ratio of `again`, `hard`, `good` and `easy` ratings for new cards: %.2f, %.2f, %.2f, %.2f"
    % tuple(first_rating_prob)
)



average time for failed reviews: -0.0s
average time for recalled reviews: 0.0s
average time for `hard`, `good` and `easy` reviews: 0.0s, 0.0s, -0.0s
average time for learning a new card: 0.0s
Ratio of `hard`, `good` and `easy` ratings for recalled reviews: 0.33, 0.00, 0.67
Ratio of `again`, `hard`, `good` and `easy` ratings for new cards: 0.29, 0.29, 0.15, 0.26


In [189]:
df.head()

,id,review_time,review_state,z_scores,review_duration,review_rating,review_time_curr,review_state_curr
0,1,"[1, 5, 10, 15]","[0, 1, 1, 1]",1.0,5,1,15,1
1,2,"[20, 25, 30]","[3, 2, 1]",0.3,5,3,30,1
2,3,[5],[2],0.5,5,2,5,2
3,4,"[3, 7]","[0, 1]",0.7,5,2,7,1
4,5,"[2, 6, 10, 14]","[1, 2, 3, 2]",0.9,4,1,14,2


In [190]:
from fsrs_optimizer import lineToTensor, FSRS
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

date = 30

def generate_rating(review_type):
    if review_type == "new":
        return np.random.choice([1, 2, 3, 4], p=first_rating_prob)
    elif review_type == "recall":
        return np.random.choice([2, 3, 4], p=review_rating_prob)

class Collection:
    def __init__(self):
        self.model = FSRS(w)
        self.model.eval()

    def states(self, t_history, r_history):
        with torch.no_grad():
            line_tensor = lineToTensor(
                list(zip([str(t_history)], [str(r_history)]))[0]
            ).unsqueeze(1)
            output_t = self.model(line_tensor)
            return output_t[-1][0]

    def next_states(self, states, t, r):
        with torch.no_grad():
            return self.model.step(torch.FloatTensor([[t, r]]), states.unsqueeze(0))[0]

    def init(self, idx):
        t = df["review_time_curr"][idx]
        r = df["review_rating"][idx]
        p = round(first_rating_prob[r - 1], 2)
        new_states = self.states(t, r)
        return r, t, p, new_states


feature_list = [
    "id",
    "difficulty",
    "stability",
    "retrievability",
    "delta_t",
    "reps",
    "lapses",
    "last_date",
    "due",
    "r_history",
    "t_history",
    "p_history",
    "states",
    "time",
    "factor",
]
field_map = {key: i for i, key in enumerate(feature_list)}


def fsrs4anki_scheduler(stability):
    def constrain_interval(stability):
        if stability > 0:
            return min(
                max(1, round(9 * stability * (1 / requestRetention - 1))),
                maximumInterval,
            )
        else:
            return 1

    interval = constrain_interval(stability)
    return interval


def scheduler(fsrs_inputs):
        return fsrs4anki_scheduler(fsrs_inputs), 2.5

#for scheduler_name in ("anki", "fsrs"):
for scheduler_name in ["fsrs"]:
    new_card_per_day = np.array([0] * learn_days)
    new_card_per_day_average_per_period = np.array([0.0] * learn_days)
    review_card_per_day = np.array([0.0] * learn_days)
    review_card_per_day_average_per_period = np.array([0.0] * learn_days)
    time_per_day = np.array([0.0] * learn_days)
    time_per_day_average_per_period = np.array([0.0] * learn_days)
    learned_per_day = np.array([0.0] * learn_days)
    retention_per_day = np.array([0.0] * learn_days)
    expected_memorization_per_day = np.array([0.0] * learn_days)

    card = pd.DataFrame(
        np.zeros((deck_size, len(feature_list))),
        index=range(deck_size),
        columns=feature_list,
    )
    card["id"] = df["id"]
    card["states"] = card["states"].astype(object)
    card['reps'] = df['review_state'].apply(lambda x: len(x))
    card["lapses"] = 0
    card["due"] = learn_days
    card["last_date"] = df["review_time"].apply(lambda x: x[-1])
    
    student = Collection()
    random.seed(2022)
    # do 1 step:
    day = date
    reviewed = 0
    learned = 0
    review_time_today = 0
    learn_time_today = 0

    card["delta_t"] = day - card["last_date"]
    card["retrievability"] = np.power(
        1 + card["delta_t"] / (9 * card["stability"]), -1
    )
    # need_review = (
    #     card[card["due"] <= day]
    #     if leechSuspend == False
    #     else card[(card["due"] <= day) & (card["lapses"] < leechThreshold)]
    # )
    # retention_per_day[day] = need_review["retrievability"].mean()
    # for idx in need_review.index:
    #     if (
    #         reviewed >= review_limits
    #         or review_time_today + learn_time_today >= max_time_limts
    #     ):
    #         break

    #     reviewed += 1
    #     last_date = card.iat[idx, field_map["last_date"]]
    #     due = card.iat[idx, field_map["due"]]
    #     factor = card.iat[idx, field_map["factor"]]
    #     card.iat[idx, field_map["last_date"]] = day
    #     ivl = card.iat[idx, field_map["delta_t"]]
    #     card.iat[idx, field_map["t_history"]] += f",{ivl}"

    #     stability = card.iat[idx, field_map["stability"]]
    #     retrievability = card.iat[idx, field_map["retrievability"]]
    #     card.iat[idx, field_map["p_history"]] += f",{retrievability:.2f}"
    #     reps = card.iat[idx, field_map["reps"]]
    #     lapses = card.iat[idx, field_map["lapses"]]
    #     states = card.iat[idx, field_map["states"]]

    #     if random.random() < retrievability:
    #         rating = generate_rating("recall")
    #         recall_time = recall_costs[rating - 2]
    #         review_time_today += recall_time
    #         card.iat[idx, field_map["r_history"]] += f",{rating}"
    #         new_states = student.next_states(states, ivl, rating)
    #         new_stability = float(new_states[0])
    #         new_difficulty = float(new_states[1])
    #         card.iat[idx, field_map["stability"]] = new_stability
    #         card.iat[idx, field_map["difficulty"]] = new_difficulty
    #         card.iat[idx, field_map["states"]] = new_states
    #         card.iat[idx, field_map["reps"]] = reps + 1
    #         card.iat[idx, field_map["time"]] += recall_time

    #         delta_t, factor = scheduler(new_stability)
    #         card.iat[idx, field_map["due"]] = day + delta_t
    #         card.iat[idx, field_map["factor"]] = factor
    #     else:
    #         review_time_today += forget_cost

    #         rating = 1
    #         card.iat[idx, field_map["r_history"]] += f",{rating}"

    #         new_states = student.next_states(states, ivl, 1)
    #         new_stability = float(new_states[0])
    #         new_difficulty = float(new_states[1])

    #         card.iat[idx, field_map["stability"]] = new_stability
    #         card.iat[idx, field_map["difficulty"]] = new_difficulty
    #         card.iat[idx, field_map["states"]] = new_states

    #         reps = 0
    #         lapses = lapses + 1

    #         card.iat[idx, field_map["reps"]] = reps
    #         card.iat[idx, field_map["lapses"]] = lapses
    #         delta_t, factor = scheduler(new_stability)
    #         card.iat[idx, field_map["due"]] = day + delta_t
    #         card.iat[idx, field_map["factor"]] = factor

    #         card.iat[idx, field_map["time"]] += forget_cost

    need_learn = card[card["stability"] == 0]

    for idx in need_learn.index:
        if (
            learned >= new_cards_limits
            or review_time_today + learn_time_today >= max_time_limts
        ):
            break
        learned += 1
        learn_time_today += learn_cost
        #card.iat[idx, field_map["last_date"]] = day

        #card.iat[idx, field_map["reps"]] = 1
        #card.iat[idx, field_map["lapses"]] = 0

        r, t, p, new_states = student.init(idx)
        new_stability = float(new_states[0])
        new_difficulty = float(new_states[1])
        card['r_history'] = card['r_history'].astype(object)
        card['t_history'] = card['t_history'].astype(object)
        card['p_history'] = card['p_history'].astype(object)
        card.iat[idx, field_map["r_history"]] = str(r)
        card.iat[idx, field_map["t_history"]] = str(t)
        card.iat[idx, field_map["p_history"]] = str(p)
        card.iat[idx, field_map["stability"]] = new_stability
        card.iat[idx, field_map["difficulty"]] = new_difficulty
        card.iat[idx, field_map["states"]] = new_states

        delta_t, factor = scheduler(new_stability)
        card.iat[idx, field_map["due"]] = day + delta_t
        #card.iat[idx, field_map["due"]] = day + delta_t
        card.iat[idx, field_map["factor"]] = factor

        card.iat[idx, field_map["time"]] = learn_cost


    new_card_per_day[day] = learned
    review_card_per_day[day] = reviewed
    learned_per_day[day] = learned_per_day[day - 1] + learned
    time_per_day[day] = review_time_today + learn_time_today
    expected_memorization_per_day[day] = sum(
        card[card["retrievability"] > 0]["retrievability"]
    )

    if day >= moving_average_period:
        new_card_per_day_average_per_period[day] = np.true_divide(
            new_card_per_day[day - moving_average_period: day].sum(),
            moving_average_period,
        )
        review_card_per_day_average_per_period[day] = np.true_divide(
            review_card_per_day[day - moving_average_period: day].sum(),
            moving_average_period,
        )
        time_per_day_average_per_period[day] = np.true_divide(
            time_per_day[day - moving_average_period: day].sum(),
            moving_average_period,
        )
    else:
        new_card_per_day_average_per_period[day] = np.true_divide(
            new_card_per_day[: day + 1].sum(), day + 1
        )
        review_card_per_day_average_per_period[day] = np.true_divide(
            review_card_per_day[: day + 1].sum(), day + 1
        )
        time_per_day_average_per_period[day] = np.true_divide(
            time_per_day[: day + 1].sum(), day + 1
        )

total_learned = sum(new_card_per_day)
total_time = sum(time_per_day)
total_remembered = int(card["retrievability"].sum())
total_leeches = len(card[card["lapses"] >= leechThreshold])


In [191]:
card

,id,difficulty,stability,retrievability,delta_t,reps,lapses,last_date,due,r_history,t_history,p_history,states,time,factor
0,1,8.4740,1.1008,0.0,15,4,0,15,32,1,15,0.29,"[tensor(1.1008), tensor(8.4740)]",0.0,2.5
1,2,5.3148,5.7619,NaN,0,3,0,30,41,3,30,0.15,"[tensor(5.7619), tensor(5.3148)]",0.0,2.5
2,3,6.8944,1.2746,0.0,25,1,0,5,33,2,5,0.29,"[tensor(1.2746), tensor(6.8944)]",0.0,2.5
3,4,6.8944,1.2746,0.0,23,2,0,7,33,2,7,0.29,"[tensor(1.2746), tensor(6.8944)]",0.0,2.5
4,5,8.4740,1.1008,0.0,16,4,0,14,32,1,14,0.29,"[tensor(1.1008), tensor(8.4740)]",0.0,2.5
5,6,3.7352,10.5114,0.0,25,3,0,5,51,4,5,0.26,"[tensor(10.5114), tensor(3.7352)]",0.0,2.5
6,7,3.7352,10.5114,0.0,24,3,0,6,51,4,6,0.26,"[tensor(10.5114), tensor(3.7352)]",0.0,2.5
7,8,8.4740,1.1008,0.0,13,4,0,17,32,1,17,0.29,"[tensor(1.1008), tensor(8.4740)]",0.0,2.5
8,9,5.3148,5.7619,0.0,18,1,0,12,41,3,12,0.15,"[tensor(5.7619), tensor(5.3148)]",0.0,2.5
9,10,5.3148,5.7619,0.0,12,1,0,18,41,3,18,0.15,"[tensor(5.7619), tensor(5.3148)]",0.0,2.5


In [192]:
scaler = MinMaxScaler(feature_range=(0, 1))
card['schedule_score'] = scaler.fit_transform(card[['due']])

# Inverting the values so that lower 'due' values are closer to 1
card['schedule_score'] = 1 - card['schedule_score']

# Returning the DataFrame with 'id' and 'schedule_score'
result = card[['id', 'schedule_score']]
result

,id,schedule_score
0,1,1.000000
1,2,0.526316
2,3,0.947368
3,4,0.947368
4,5,1.000000
5,6,0.000000
6,7,0.000000
7,8,1.000000
8,9,0.526316
9,10,0.526316


In [195]:
import get_schedule
result_test = get_schedule.get_schedule_scores(df, 30)

In [196]:
result_test

,id,schedule_score
0,1,1.000000
1,2,0.526316
2,3,0.947368
3,4,0.947368
4,5,1.000000
5,6,0.000000
6,7,0.000000
7,8,1.000000
8,9,0.526316
9,10,0.526316


In [193]:
# Here feature_list doesn't have id cause we don't care which question it is for simulation
feature_list = [
    "difficulty",
    "stability",
    "retrievability",
    "delta_t",
    "reps",
    "lapses",
    "last_date",
    "due",
    "r_history",
    "t_history",
    "p_history",
    "states",
    "time",
    "factor",
]
for scheduler_name in ("anki", "fsrs"):
    new_card_per_day = np.array([0] * learn_days)
    new_card_per_day_average_per_period = np.array([0.0] * learn_days)
    review_card_per_day = np.array([0.0] * learn_days)
    review_card_per_day_average_per_period = np.array([0.0] * learn_days)
    time_per_day = np.array([0.0] * learn_days)
    time_per_day_average_per_period = np.array([0.0] * learn_days)
    learned_per_day = np.array([0.0] * learn_days)
    retention_per_day = np.array([0.0] * learn_days)
    expected_memorization_per_day = np.array([0.0] * learn_days)

    card = pd.DataFrame(
        np.zeros((deck_size, len(feature_list))),
        index=range(deck_size),
        columns=feature_list,
    )
    card["states"] = card["states"].astype(object)
    card['reps'] = df['review_state'].apply(lambda x: len(x))
    card["lapses"] = 0
    card["due"] = learn_days
    card["last_date"] = df["review_time"].apply(lambda x: x[-1])
    
    student = Collection()
    random.seed(2022)
    for day in tqdm(range(learn_days)):
        reviewed = 0
        learned = 0
        review_time_today = 0
        learn_time_today = 0

        card["delta_t"] = day - card["last_date"]
        card["retrievability"] = np.power(
            1 + card["delta_t"] / (9 * card["stability"]), -1
        )
        need_review = (
            card[card["due"] <= day]
            if leechSuspend == False
            else card[(card["due"] <= day) & (card["lapses"] < leechThreshold)]
        )
        retention_per_day[day] = need_review["retrievability"].mean()
        for idx in need_review.index:
            if (
                reviewed >= review_limits
                or review_time_today + learn_time_today >= max_time_limts
            ):
                break

            reviewed += 1
            last_date = card.iat[idx, field_map["last_date"]]
            due = card.iat[idx, field_map["due"]]
            factor = card.iat[idx, field_map["factor"]]
            card.iat[idx, field_map["last_date"]] = day
            ivl = card.iat[idx, field_map["delta_t"]]
            card.iat[idx, field_map["t_history"]] += f",{ivl}"

            stability = card.iat[idx, field_map["stability"]]
            retrievability = card.iat[idx, field_map["retrievability"]]
            card.iat[idx, field_map["p_history"]] += f",{retrievability:.2f}"
            reps = card.iat[idx, field_map["reps"]]
            lapses = card.iat[idx, field_map["lapses"]]
            states = card.iat[idx, field_map["states"]]

            if random.random() < retrievability:
                rating = generate_rating("recall")
                recall_time = recall_costs[rating - 2]
                review_time_today += recall_time
                card.iat[idx, field_map["r_history"]] += f",{rating}"
                new_states = student.next_states(states, ivl, rating)
                new_stability = float(new_states[0])
                new_difficulty = float(new_states[1])
                card.iat[idx, field_map["stability"]] = new_stability
                card.iat[idx, field_map["difficulty"]] = new_difficulty
                card.iat[idx, field_map["states"]] = new_states
                card.iat[idx, field_map["reps"]] = reps + 1
                card.iat[idx, field_map["time"]] += recall_time

                delta_t, factor = scheduler(
                    scheduler_name,
                    new_stability,
                    (due - last_date, ivl, factor, rating),
                )
                card.iat[idx, field_map["factor"]] = factor
                card.iat[idx, field_map["due"]] = day + delta_t

            else:
                review_time_today += forget_cost

                rating = 1
                card.iat[idx, field_map["r_history"]] += f",{rating}"

                new_states = student.next_states(states, ivl, 1)
                new_stability = float(new_states[0])
                new_difficulty = float(new_states[1])

                card.iat[idx, field_map["stability"]] = new_stability
                card.iat[idx, field_map["difficulty"]] = new_difficulty
                card.iat[idx, field_map["states"]] = new_states

                reps = 0
                lapses = lapses + 1

                card.iat[idx, field_map["reps"]] = reps
                card.iat[idx, field_map["lapses"]] = lapses

                delta_t, factor = scheduler(
                    scheduler_name,
                    new_stability,
                    (due - last_date, ivl, factor, rating),
                )
                card.iat[idx, field_map["due"]] = day + delta_t
                card.iat[idx, field_map["factor"]] = factor
                card.iat[idx, field_map["time"]] += forget_cost

        need_learn = card[card["stability"] == 0]

        for idx in need_learn.index:
            if (
                learned >= new_cards_limits
                or review_time_today + learn_time_today >= max_time_limts
            ):
                break
            learned += 1
            learn_time_today += learn_cost
            card.iat[idx, field_map["last_date"]] = day

            card.iat[idx, field_map["reps"]] = 1
            card.iat[idx, field_map["lapses"]] = 0

            r, t, p, new_states = student.init()
            new_stability = float(new_states[0])
            new_difficulty = float(new_states[1])

            card.iat[idx, field_map["r_history"]] = str(r)
            card.iat[idx, field_map["t_history"]] = str(t)
            card.iat[idx, field_map["p_history"]] = str(p)
            card.iat[idx, field_map["stability"]] = new_stability
            card.iat[idx, field_map["difficulty"]] = new_difficulty
            card.iat[idx, field_map["states"]] = new_states

            delta_t, factor = scheduler(
                scheduler_name, new_stability, (None, None, None, r)
            )
            card.iat[idx, field_map["due"]] = day + delta_t
            card.iat[idx, field_map["time"]] = learn_cost
            card.iat[idx, field_map["factor"]] = factor

        new_card_per_day[day] = learned
        review_card_per_day[day] = reviewed
        learned_per_day[day] = learned_per_day[day - 1] + learned
        time_per_day[day] = review_time_today + learn_time_today
        expected_memorization_per_day[day] = sum(
            card[card["retrievability"] > 0]["retrievability"]
        )

        if day >= moving_average_period:
            new_card_per_day_average_per_period[day] = np.true_divide(
                new_card_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
            review_card_per_day_average_per_period[day] = np.true_divide(
                review_card_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
            time_per_day_average_per_period[day] = np.true_divide(
                time_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
        else:
            new_card_per_day_average_per_period[day] = np.true_divide(
                new_card_per_day[: day + 1].sum(), day + 1
            )
            review_card_per_day_average_per_period[day] = np.true_divide(
                review_card_per_day[: day + 1].sum(), day + 1
            )
            time_per_day_average_per_period[day] = np.true_divide(
                time_per_day[: day + 1].sum(), day + 1
            )

    total_learned = sum(new_card_per_day)
    total_time = sum(time_per_day)
    total_remembered = int(card["retrievability"].sum())
    total_leeches = len(card[card["lapses"] >= leechThreshold])

    plt.figure(1)
    plt.plot(review_card_per_day_average_per_period, label=f"{scheduler_name}")
    plt.figure(2)
    plt.plot(time_per_day_average_per_period / 60, label=f"{scheduler_name}")
    plt.figure(3)
    plt.plot(learned_per_day, label=f"{scheduler_name}")
    plt.figure(4)
    plt.plot(retention_per_day, label=f"{scheduler_name}")
    plt.figure(5)
    plt.plot(expected_memorization_per_day, label=f"{scheduler_name}")

    print("scheduler:", scheduler_name)
    print("learned cards:", total_learned)
    print("time in minutes:", round(total_time / 60, 1))
    print("remembered cards:", total_remembered)
    print("time per remembered card:", round(
        total_time / 60 / total_remembered, 2))
    print("leeches:", total_leeches)

    save = card[card["retrievability"] > 0].copy()
    save["stability"] = round(save["stability"], 2)
    save["retrievability"] = round(save["retrievability"], 2)
    save["difficulty"] = round(save["difficulty"], 2)
    save["factor"] = round(save["factor"], 2)
    save["time"] = round(save["time"], 2)

    save.to_csv(f"./simulator-{scheduler_name}.csv", index=False, sep="\t")

plt.figure(1)
plt.title(f"new cards limits:{new_cards_limits}-learn days:{learn_days}")
plt.xlabel("days")
plt.ylabel(f"review cards per day ({moving_average_period} days average)")
plt.legend()
plt.grid(True)
plt.figure(2)
plt.title(f"new cards limits:{new_cards_limits}-learn days:{learn_days}")
plt.xlabel("days")
plt.ylabel(f"time in minutes per day ({moving_average_period} days average)")
plt.legend()
plt.grid(True)
plt.figure(3)
plt.title(f"new cards limits:{new_cards_limits}-learn days:{learn_days}")
plt.xlabel("days")
plt.ylabel(f"cards total learned")
plt.legend()
plt.grid(True)
plt.figure(4)
plt.title(f"new cards limits:{new_cards_limits}-learn days:{learn_days}")
plt.xlabel("days")
plt.ylabel("retention per day")
plt.legend()
plt.grid(True)
plt.figure(5)
plt.title(f"new cards limits:{new_cards_limits}-learn days:{learn_days}")
plt.xlabel("days")
plt.ylabel("expected memorization per day")
plt.legend()
plt.grid(True)
plt.show()
plt.close("all")

  0%|          | 0/50 [00:00<?, ?it/s]


TypeError: Collection.init() missing 1 required positional argument: 'idx'

In [ ]:
card.head()

,difficulty,stability,retrievability,delta_t,reps,lapses,last_date,due,r_history,t_history,p_history,states,time,factor
0,5.314800,28.385338,0.870514,38,2,0,11,67,"3,3","0,11","0.53,0.82","[tensor(28.3853), tensor(5.3148)]",0.005,2.5
1,4.074532,50.033703,0.922180,38,2,0,11,110,"3,4","0,11","0.53,0.82","[tensor(50.0337), tensor(4.0745)]",0.004,2.5
2,9.985944,5.397592,0.941836,3,1,2,46,57,"2,3,1,4,1,3","0,3,14,5,18,6","0.16,0.79,0.82,0.82,0.82,0.82","[tensor(5.3976), tensor(9.9859)]",0.014,2.5
3,7.205385,44.429798,0.954640,19,4,0,30,118,"1,3,3,4","0,2,7,21","0.11,0.83,0.82,0.82","[tensor(44.4298), tensor(7.2054)]",0.014,2.5
4,2.499671,107.331604,0.971831,28,2,0,21,233,"4,4","0,21","0.21,0.82","[tensor(107.3316), tensor(2.4997)]",0.004,2.5
